<div style="background-color:#000;"><img src="pqn.png"></img></div>

This code cross-validates a parameterized trading strategy using historical data. It defines a cross-validation schema that splits data into training and testing sets based on specified time ranges. The code then applies a simple trading strategy, an EMA crossover with an ATR trailing stop, to each split. It evaluates the strategy's performance using the Sharpe ratio and performs parameter optimization to test various combinations. Finally, it analyzes the correlation between training and testing results to assess the strategy's robustness.

In [ ]:
import numpy as np
from pandas.tseries.frequencies import to_offset
import vectorbtpro as vbt

Set the theme for VectorBT plots to dark

In [ ]:
vbt.settings.set_theme("dark")

Define parameters for the data pull, including symbol, start and end dates, and timeframe

In [ ]:
SYMBOL = "AAPL"
START = "2010"
END = "now"
TIMEFRAME = "day"

Pull historical data for the specified symbol and timeframe

In [ ]:
data = vbt.YFData.pull(
    SYMBOL,
    start=START,
    end=END,
    timeframe=TIMEFRAME
)

Define parameters for the cross-validation schema, including training and testing periods

In [ ]:
TRAIN = 12
TEST = 12
EVERY = 3
OFFSET = "MS"

Create a splitter object that divides the date range into training and testing sets

In [ ]:
splitter = vbt.Splitter.from_ranges(
    data.index, 
    every=f"{EVERY}{OFFSET}", 
    lookback_period=f"{TRAIN + TEST}{OFFSET}",
    split=(
        vbt.RepFunc(lambda index: index < index[0] + TRAIN * to_offset(OFFSET)),
        vbt.RepFunc(lambda index: index >= index[0] + TRAIN * to_offset(OFFSET)),
    ),
    set_labels=["train", "test"]
)

Display the splitter plots to visualize the training and testing sets

In [ ]:
splitter.plots().show_png()

Define an objective function to execute a trading strategy with specific parameters

In [ ]:
def objective(data, fast_period=10, slow_period=20, atr_period=14, atr_mult=3):
    """Execute EMA crossover with ATR trailing stop
    
    Parameters
    ----------
    data : vbt.Data
        Historical price data
    fast_period : int, optional
        Period for fast EMA, by default 10
    slow_period : int, optional
        Period for slow EMA, by default 20
    atr_period : int, optional
        Period for ATR, by default 14
    atr_mult : int, optional
        Multiplier for ATR trailing stop, by default 3
    
    Returns
    -------
    float
        Sharpe ratio of the strategy
    """
    
    # Calculate fast and slow EMAs and ATR for the given periods
    fast_ema = data.run("talib:ema", fast_period, short_name="fast_ema", unpack=True)
    slow_ema = data.run("talib:ema", slow_period, short_name="slow_ema", unpack=True)
    atr = data.run("talib:atr", atr_period, unpack=True)
    
    # Define a portfolio using EMA crossover signals and ATR trailing stop
    pf = vbt.PF.from_signals(
        data, 
        entries=fast_ema.vbt.crossed_above(slow_ema), 
        exits=fast_ema.vbt.crossed_below(slow_ema), 
        tsl_stop=atr * atr_mult, 
        save_returns=True,
        freq=TIMEFRAME
    )
    
    # Return the Sharpe ratio of the portfolio
    return pf.sharpe_ratio

Print the Sharpe ratio for the objective function with default parameters

In [ ]:
print(objective(data))

Decorate the objective function to enable it to accept lists of parameters and execute across combinations

In [ ]:
param_objective = vbt.parameterized(
    objective,
    merge_func="concat",
    mono_n_chunks="auto",
    execute_kwargs=dict(engine="pathos")
)

Further decorate the function to run across date ranges specified by the splitter

In [ ]:
cv_objective = vbt.split(
    param_objective,
    splitter=splitter, 
    takeable_args=["data"], 
    merge_func="concat", 
    execute_kwargs=dict(show_progress=True)
)

Generate Sharpe ratio results for various parameter combinations using cross-validation

In [ ]:
sharpe_ratio = cv_objective(
    data,
    vbt.Param(np.arange(10, 50), condition="slow_period - fast_period >= 5"),
    vbt.Param(np.arange(10, 50)),
    vbt.Param(np.arange(10, 50), condition="fast_period <= atr_period <= slow_period"),
    vbt.Param(np.arange(2, 5))
)

Print the resulting Sharpe ratio for the parameter combinations

In [ ]:
print(sharpe_ratio)

Extract the Sharpe ratio for the training set

In [ ]:
train_sharpe_ratio = sharpe_ratio.xs("train", level="set")

Extract the Sharpe ratio for the testing set

In [ ]:
test_sharpe_ratio = sharpe_ratio.xs("test", level="set")

Print the correlation between training and testing Sharpe ratios

In [ ]:
print(train_sharpe_ratio.corr(test_sharpe_ratio))

Calculate the difference in Sharpe ratios between testing and training sets

In [ ]:
sharpe_ratio_diff = test_sharpe_ratio - train_sharpe_ratio

Compute the median difference in Sharpe ratios grouped by fast and slow EMA periods

In [ ]:
sharpe_ratio_diff_median = sharpe_ratio_diff.groupby(
    ["fast_period", "slow_period"]
).median()

Display a heatmap of the median differences

In [ ]:
sharpe_ratio_diff_median.vbt.heatmap(
    trace_kwargs=dict(colorscale="RdBu")
).show_png()

<a href="https://pyquantnews.com/">PyQuant News</a> is where finance practitioners level up with Python for quant finance, algorithmic trading, and market data analysis. Looking to get started? Check out the fastest growing, top-selling course to <a href="https://gettingstartedwithpythonforquantfinance.com/">get started with Python for quant finance</a>. For educational purposes. Not investment advise. Use at your own risk.